In [1]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [2]:
from evaluations.performance_01 import get_per_sample_metrics
metrics_df = get_per_sample_metrics()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
metrics_df.head()

,seed,model,checkpoint,dataset,doc_id,targets,entity_type,tp,fn,fp,error_type1,error_type2,error_type3,error_type4,error_type5
0,2,t5_asp_lownergaz,best,lowner_dev,lowner_dev_0,0,location,0,0,0,0,0,0,0,0
1,2,t5_asp_lownergaz,best,lowner_dev,lowner_dev_0,0,group,0,0,0,0,0,0,0,0
2,2,t5_asp_lownergaz,best,lowner_dev,lowner_dev_0,0,corporation,0,0,0,0,0,0,0,0
3,2,t5_asp_lownergaz,best,lowner_dev,lowner_dev_0,0,person,0,0,0,0,0,0,0,0
4,2,t5_asp_lownergaz,best,lowner_dev,lowner_dev_0,0,creative-work,0,0,0,0,0,0,0,0


In [3]:
metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10897200 entries, 0 to 10897199
Data columns (total 15 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   seed         object
 1   model        object
 2   checkpoint   object
 3   dataset      object
 4   doc_id       object
 5   targets      int64 
 6   entity_type  object
 7   tp           int64 
 8   fn           int64 
 9   fp           int64 
 10  error_type1  int64 
 11  error_type2  int64 
 12  error_type3  int64 
 13  error_type4  int64 
 14  error_type5  int64 
dtypes: int64(9), object(6)
memory usage: 1.2+ GB


In [4]:
metrics_df.describe()

,targets,tp,fn,fp,error_type1,error_type2,error_type3,error_type4,error_type5
count,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07,1.089720e+07
mean,2.161282e-01,1.905645e-01,2.556363e-02,1.679248e-02,5.740924e-03,1.422549e-02,4.029292e-03,1.301343e-03,5.957035e-03
std,5.983969e-01,5.668767e-01,1.740430e-01,1.412545e-01,8.191047e-02,1.271610e-01,6.762273e-02,3.704992e-02,8.482927e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.200000e+01,1.200000e+01,8.000000e+00,7.000000e+00,7.000000e+00,8.000000e+00,6.000000e+00,3.000000e+00,5.000000e+00


# Model performance
## Macro-F1 + 5 Error Types avg+std

In [6]:
# Macro - F1
from evaluations.performance_01 import get_f1

get_f1(metrics_df, "last", "lowner_test").sort_values(("f1", "mean"), ascending=False)

model checkpoint      dataset         f1          
                                                        mean       std
34  t5_asp_lownergaz_sent       last  lowner_test  89.600103  0.029477
22        t5_asp_gaz_sent       last  lowner_test  88.478024  0.122478
28       t5_asp_lownergaz       last  lowner_test  87.478254  2.444407
40            t5_asp_sent       last  lowner_test  86.724683  0.218033
16             t5_asp_gaz       last  lowner_test  86.515093  1.373751
10                 t5_asp       last  lowner_test  85.481521  0.163211
4           flair_roberta       last  lowner_test  84.064658  0.110460

In [3]:
from evaluations.performance_01 import get_recall

get_recall(metrics_df, "last", "lowner_test").sort_values(("recall", "mean"),
                                                          ascending=False)


model checkpoint      dataset     recall          
                                                        mean       std
34  t5_asp_lownergaz_sent       last  lowner_test  88.386403  0.255862
22        t5_asp_gaz_sent       last  lowner_test  87.440906  0.310630
4           flair_roberta       last  lowner_test  85.186202  0.112832
28       t5_asp_lownergaz       last  lowner_test  84.831641  4.507486
40            t5_asp_sent       last  lowner_test  84.766828  0.357262
16             t5_asp_gaz       last  lowner_test  84.359655  2.830382
10                 t5_asp       last  lowner_test  84.031781  0.061052

In [7]:
# 5 Error types
from evaluations.performance_01 import get_error_types
get_error_types(metrics_df, "last", "lowner_test")

model checkpoint      dataset  error_type1              \
                                                          mean         std   
4           flair_roberta       last  lowner_test  2933.666667  103.345698   
10                 t5_asp       last  lowner_test  1652.666667   71.514567   
16             t5_asp_gaz       last  lowner_test  1618.000000  146.447943   
22        t5_asp_gaz_sent       last  lowner_test  1670.000000   93.642939   
28       t5_asp_lownergaz       last  lowner_test  1519.000000  148.704405   
34  t5_asp_lownergaz_sent       last  lowner_test  1577.000000  111.139552   
40            t5_asp_sent       last  lowner_test  1473.000000   85.281886   

    error_type2               error_type3            error_type4             \
           mean          std         mean        std        mean        std   
4   2118.000000   112.680966  1527.000000  25.632011  683.000000  26.851443   
10  3078.000000    60.307545  1595.000000   4.582576  492.000000  16.370706   
16  3744.000000  1384.099707  1078.333333  80.164414  362.000000  19.697716   
22  2588.666667    99.006734   985.000000  21.702534  341.333333  11.930353   
28  4099.000000  2043.395948   908.666667  27.970222  242.666667  13.051181   
34  2622.333333   121.705930   855.666667   6.506407  234.333333   1.527525   
40  3334.000000   134.357731  1235.000000  16.822604  402.000000   7.000000   

    error_type5             
           mean        std  
4   2550.666667  33.171273  
10  1840.666667  56.518434  
16  1646.666667  53.947505  
22  1535.000000  17.776389  
28  1321.333333  26.312228  
34  1277.666667  38.083242  
40  1655.333333  12.858201

## Macro F1 + errors by entity type

In [7]:
from evaluations.performance_01 import get_f1_per_entity

get_f1_per_entity(metrics_df, "last", "lowner_test")

entity_type                  model checkpoint      dataset corporation  \
                                                                  mean   
                                                                    f1   
4                    flair_roberta       last  lowner_test   70.742816   
10                          t5_asp       last  lowner_test   75.113433   
16                      t5_asp_gaz       last  lowner_test   81.074021   
22                 t5_asp_gaz_sent       last  lowner_test   83.985300   
28                t5_asp_lownergaz       last  lowner_test   81.709624   
34           t5_asp_lownergaz_sent       last  lowner_test   84.335537   
40                     t5_asp_sent       last  lowner_test   79.699732   

entity_type           creative-work                group             location  \
                  std          mean       std       mean       std       mean   
                   f1            f1        f1         f1        f1         f1   
4            0.430478     66.998527  0.058221  78.793575  0.167154  89.354230   
10           0.462709     71.650596  0.102965  79.986712  0.173166  87.925792   
16           1.633502     74.033896  2.360488  84.033400  0.855054  88.329838   
22           0.400360     77.602961  0.288917  85.917129  0.374108  89.416562   
28           1.615418     77.102966  3.456147  85.925062  1.160642  89.815369   
34           0.382686     80.932509  0.187027  87.375040  0.112857  90.412207   
40           0.487246     74.173166  0.369699  82.780910  0.455662  88.244192   

entity_type               person              product            
                  std       mean       std       mean       std  
                   f1         f1        f1         f1        f1  
4            0.098830  91.635072  0.259711  50.639928  0.969308  
10           0.274731  94.275321  0.030955  44.775935  0.521565  
16           0.105124  93.215122  2.712527  51.427142  3.241317  
22           0.215385  95.178446  0.118025  61.103393  1.173573  
28           0.153259  92.551586  5.597763  50.420167  2.201350  
34           0.064019  95.841276  0.124411  59.474805  0.588127  
40           0.154587  94.552950  0.113219  53.592878  0.824031

In [3]:
from evaluations.performance_01 import get_error_types_per_entity

get_error_types_per_entity(metrics_df, "last", "lowner_test")

entity_type                  model checkpoint      dataset corporation  \
                                                                  mean   
                                                           error_type1   
4                           t5_asp       last  lowner_test   75.333333   
10                      t5_asp_gaz       last  lowner_test   98.666667   
16                 t5_asp_gaz_sent       last  lowner_test   88.666667   
22                t5_asp_lownergaz       last  lowner_test   81.000000   
28           t5_asp_lownergaz_sent       last  lowner_test   87.000000   
34                     t5_asp_sent       last  lowner_test   76.000000   

entity_type             creative-work                   group              \
                    std          mean         std        mean         std   
            error_type1   error_type1 error_type1 error_type1 error_type1   
4              4.725816    415.000000   15.132746  233.000000    8.660254   
10             4.163332    414.333333    4.041452  205.333333   25.501634   
16             6.658328    458.000000   24.515301  208.666667    9.237604   
22             7.211103    377.666667   53.425961  185.666667   21.594752   
28             2.645751    385.333333   15.567059  186.666667   23.158872   
34             3.464102    396.666667   30.353473  191.333333   20.599353   

entity_type    location  ... creative-work                   group  \
                   mean  ...          mean         std        mean   
            error_type1  ...   error_type5 error_type5 error_type5   
4            375.666667  ...    326.666667   11.676187  304.333333   
10           356.666667  ...    258.666667   10.016653  258.666667   
16           387.000000  ...    273.666667   18.770544  243.666667   
22           359.000000  ...    198.000000   13.453624  211.333333   
28           369.666667  ...    182.333333   16.921387  200.000000   
34           315.333333  ...    269.000000   10.000000  277.666667   

entity_type                location                  person              \
                    std        mean         std        mean         std   
            error_type5 error_type5 error_type5 error_type5 error_type5   
4              8.020806  937.000000   45.574115  173.666667    2.886751   
10            22.479620  892.333333   28.746014  172.333333    8.082904   
16             4.163332  843.000000    2.000000  166.666667   14.011900   
22            12.858201  700.333333   17.616280  128.000000    7.810250   
28             5.291503  682.333333   11.930353  129.000000    7.549834   
34             5.131601  855.666667   18.475209  169.000000    8.544004   

entity_type     product              
                   mean         std  
            error_type5 error_type5  
4             43.000000    5.000000  
10            28.000000    4.000000  
16            31.333333    0.577350  
22            31.666667    0.577350  
28            32.666667    5.859465  
34            32.333333    2.081666  

[6 rows x 63 columns]

## EECR + ECR for labeled data + search results

In [3]:
# EECR - labeled data
from evaluations.performance_01 import get_labeled_data_eecr

labeled_data_eecr_df = get_labeled_data_eecr()
labeled_data_eecr_df.sort_values("eecr", ascending=False)


EECR: 100%|██████████| 15/15 [00:55<00:00,  3.68s/it]


,model,dataset,eecr
7,t5_asp_lownergaz_sent,lowner_dev,0.871725
6,t5_asp_lownergaz_sent,lowner_train,0.871400
8,t5_asp_lownergaz_sent,lowner_test,0.869790
0,t5_asp_lownergaz,lowner_train,0.838219
2,t5_asp_lownergaz,lowner_test,0.834924
1,t5_asp_lownergaz,lowner_dev,0.832973
14,t5_asp_sent,lowner_test,0.487843
11,t5_asp_gaz_sent,lowner_test,0.486827
12,t5_asp_sent,lowner_train,0.484656
13,t5_asp_sent,lowner_dev,0.484016


In [3]:
# EECR - labeled data - per sample eecr
from evaluations.performance_01 import get_labeled_data_eecr_per_sample

labeled_data_eecr_sample_df = get_labeled_data_eecr_per_sample()
labeled_data_eecr_sample_df.sort_values("eecr", ascending=False)

EECR labeled data per sample: 100%|██████████| 15/15 [00:47<00:00,  3.15s/it]


,model,dataset,eecr
10,t5_asp_lownergaz_sent,lowner_test,0.732972
11,t5_asp_lownergaz_sent,lowner_train,0.729629
9,t5_asp_lownergaz_sent,lowner_dev,0.729129
7,t5_asp_lownergaz,lowner_test,0.705637
8,t5_asp_lownergaz,lowner_train,0.703499
6,t5_asp_lownergaz,lowner_dev,0.700680
13,t5_asp_sent,lowner_test,0.396824
4,t5_asp_gaz_sent,lowner_test,0.395983
14,t5_asp_sent,lowner_train,0.393449
5,t5_asp_gaz_sent,lowner_train,0.392618


In [3]:
# EECR - search results - per sample eecr
from evaluations.performance_01 import get_search_results_data_eecr

search_results_data_eecr = get_search_results_data_eecr()
search_results_data_eecr.sort_values("eecr", ascending=False)


EECR: 100%|██████████| 15/15 [00:08<00:00,  1.84it/s]


,model,dataset,eecr
10,t5_asp_lownergaz_sent,lowner_test,0.537117
9,t5_asp_lownergaz_sent,lowner_dev,0.535160
11,t5_asp_lownergaz_sent,lowner_train,0.532016
7,t5_asp_lownergaz,lowner_test,0.518767
8,t5_asp_lownergaz,lowner_train,0.509690
6,t5_asp_lownergaz,lowner_dev,0.508774
4,t5_asp_gaz_sent,lowner_test,0.333394
5,t5_asp_gaz_sent,lowner_train,0.331374
3,t5_asp_gaz_sent,lowner_dev,0.325894
1,t5_asp_gaz,lowner_test,0.321917


In [4]:
# ECR - labeled data - 5 classes
from evaluations.performance_01 import get_labeled_data_ecr

labeled_data_ecr_df = get_labeled_data_ecr()

labeled_data_ecr_df

ECR classes labeled data: 100%|██████████| 15/15 [02:08<00:00,  8.54s/it]


,model,dataset,ρ=1,"ρ ∈ (0.5,1)","ρ ∈ (0,0.5]",ρ=0∧C≠0,ρ=0∧C=0
0,t5_asp_lownergaz,lowner_train,10742,611,862,157,1183
1,t5_asp_lownergaz,lowner_dev,2947,215,280,46,325
2,t5_asp_lownergaz,lowner_test,24403,1119,1790,360,2867
3,t5_asp_gaz,lowner_train,5060,20,309,174,7992
4,t5_asp_gaz,lowner_dev,1565,1,116,44,2087
5,t5_asp_gaz,lowner_test,10346,65,496,391,19241
6,t5_asp_lownergaz_sent,lowner_train,10760,949,545,140,1161
7,t5_asp_lownergaz_sent,lowner_dev,2953,359,141,43,317
8,t5_asp_lownergaz_sent,lowner_test,24448,1765,1180,339,2807
9,t5_asp_gaz_sent,lowner_train,5060,222,107,174,7992


In [3]:
from evaluations.performance_01 import get_labeled_data_ecr_per_sample

labeled_data_ecr_sample_df = get_labeled_data_ecr_per_sample()
labeled_data_ecr_sample_df

ECR classes labeled data per sample: 100%|██████████| 15/15 [00:32<00:00,  2.18s/it]


,model,dataset,"ρ ∈ (0,0.5]","ρ ∈ (0.5,1)",ρ=0∧C=0,ρ=0∧C≠0,ρ=1
0,t5_asp_gaz,lowner_dev,193,0,2138,63,1944
1,t5_asp_gaz,lowner_test,1906,0,21207,834,19727
2,t5_asp_gaz,lowner_train,805,0,8486,293,7770
3,t5_asp_gaz_sent,lowner_dev,40,153,2138,63,1944
4,t5_asp_gaz_sent,lowner_test,338,1568,21207,834,19727
5,t5_asp_gaz_sent,lowner_train,186,619,8486,293,7770
6,t5_asp_lownergaz,lowner_dev,415,336,333,51,3203
7,t5_asp_lownergaz,lowner_test,3983,3491,3214,640,32346
8,t5_asp_lownergaz,lowner_train,1511,1375,1280,225,12963
9,t5_asp_lownergaz_sent,lowner_dev,172,584,320,48,3214


In [3]:
from evaluations.performance_01 import get_search_results_data_ecr

search_results_data_ecr = get_search_results_data_ecr()
search_results_data_ecr

ECR classes search results: 100%|██████████| 15/15 [00:12<00:00,  1.25it/s]


,model,dataset,"ρ ∈ (0,0.5]","ρ ∈ (0.5,1)",ρ=0∧C=0,ρ=0∧C≠0,ρ=1
0,t5_asp_gaz,lowner_dev,89,0,2616,46,1587
1,t5_asp_gaz,lowner_test,970,0,26078,523,16103
2,t5_asp_gaz,lowner_train,411,0,10435,168,6340
3,t5_asp_gaz_sent,lowner_dev,34,27,2553,32,1692
4,t5_asp_gaz_sent,lowner_test,338,254,25531,413,17138
5,t5_asp_gaz_sent,lowner_train,141,110,10210,139,6754
6,t5_asp_lownergaz,lowner_dev,107,55,1751,26,2399
7,t5_asp_lownergaz,lowner_test,1231,514,17317,333,24279
8,t5_asp_lownergaz,lowner_train,468,225,7020,107,9534
9,t5_asp_lownergaz_sent,lowner_dev,56,36,1633,26,2587


## Context Coverage Ratio

In [2]:
# CCR search results

from evaluations.performance_01 import get_search_results_data_ccr_metrics

df = get_search_results_data_ccr_metrics()
df.head()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CCR search results:   0%|          | 0/15 [00:00<?, ?it/s]/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
CCR search results: 100%|██████████| 15/15 [12:47<00:00, 51.14s/it]


,model,dataset,doc_id,max,"φ ∈ (0.5,1]","φ ∈ (0,0.5]","φ ∈ (-0.5,0]","φ ∈ [-1,-0.5]"
0,t5_asp_lownergaz,lowner_test,lowner_test_0,0.444935,0,12,0,0
1,t5_asp_lownergaz,lowner_test,lowner_test_1,0.639595,1,11,0,0
2,t5_asp_lownergaz,lowner_test,lowner_test_2,0.318542,0,12,0,0
3,t5_asp_lownergaz,lowner_test,lowner_test_3,0.376024,0,11,1,0
4,t5_asp_lownergaz,lowner_test,lowner_test_4,0.346099,0,12,0,0


In [4]:
from evaluations.performance_01 import get_search_results_data_ccr_max

max_ccr_search_df = get_search_results_data_ccr_max()
max_ccr_search_df[max_ccr_search_df["dataset"]=="lowner_test"].sort_values(("mean", "max"), ascending=False)

,model,dataset,mean,std
,,,max,max
10,t5_asp_lownergaz_sent,lowner_test,0.639273,0.119954
4,t5_asp_gaz_sent,lowner_test,0.630119,0.119587
13,t5_asp_sent,lowner_test,0.626859,0.119601
7,t5_asp_lownergaz,lowner_test,0.468299,0.163981
1,t5_asp_gaz,lowner_test,0.420175,0.148764
